In [ ]:
'''
net.parameters():a generator
net.named_parameter():包含了名字，
for name,param in named_parameters():
这里面的param是一个<class 'torch.nn.parameter.Parameter'>对象，
所以当在init中自己初始化这种参数对象时候，也会被添加到net.paramters中
list(net[0].parameters())[0]:取第０层的ｗ参数
param.data:取数据
param.grad：取梯度
共享参数就是，在容器中传入相同的两层
'''

In [1]:
import torch
from torch import nn
from torch.nn import init

In [16]:
net = nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [3]:
x = torch.rand(2,4)
print(net(x))
print(net(x).sum())

tensor([[-0.1221],
        [-0.1221]], grad_fn=<AddmmBackward>)
tensor(-0.2442, grad_fn=<SumBackward0>)


In [4]:
print(type(net.named_parameters()))

<class 'generator'>


In [5]:
for name, param in net.named_parameters():
    print(name,param.size())

0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [6]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [7]:
for param in net.parameters():
    print(param)
for param in net[0].parameters():
    print(param)

Parameter containing:
tensor([[-0.1052,  0.1292, -0.3931, -0.2488],
        [-0.0331, -0.4442, -0.2251, -0.2343],
        [ 0.0236,  0.4007, -0.1574, -0.2705]], requires_grad=True)
Parameter containing:
tensor([-0.1867, -0.2132, -0.2968], requires_grad=True)
Parameter containing:
tensor([[0.5094, 0.2146, 0.4552]], requires_grad=True)
Parameter containing:
tensor([-0.1221], requires_grad=True)
Parameter containing:
tensor([[-0.1052,  0.1292, -0.3931, -0.2488],
        [-0.0331, -0.4442, -0.2251, -0.2343],
        [ 0.0236,  0.4007, -0.1574, -0.2705]], requires_grad=True)
Parameter containing:
tensor([-0.1867, -0.2132, -0.2968], requires_grad=True)


In [14]:
class mymodel(nn.Module):
    def __init__(self,**kwargs):
        super(mymodel,self).__init__()
        self.weight1 = nn.Parameter(torch.rand(20,20))
        self.weight2 = torch.rand(20,20)
    def foward(self,x):
        pass
n = mymodel()
for name, param in n.named_parameters():
    print(name,param.size())

weight1 torch.Size([20, 20])


In [20]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)

tensor([[ 0.3892, -0.0748, -0.1027,  0.1574],
        [-0.3803,  0.0503, -0.1482,  0.0424],
        [-0.1011,  0.0782,  0.2001,  0.3051]])


In [21]:
print(weight_0.grad)

None


In [24]:
Y=net(x)
Y.sum().backward()

In [25]:
print(weight_0.grad)#反向传播以后才会有梯度值！

tensor([[ 0.1961,  0.0059,  0.1780,  0.2327],
        [-0.6763, -0.3610, -0.5222, -0.4619],
        [-0.7704, -0.4112, -0.5948, -0.5261]])


In [27]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0,std=0.01)
        print(name,param.data)
    elif 'bias' in name:
        init.constant_(param,val=0)
        print(name,param.data)

0.weight tensor([[-0.0013, -0.0024, -0.0149,  0.0046],
        [ 0.0144, -0.0036,  0.0034,  0.0116],
        [ 0.0173,  0.0013, -0.0042, -0.0057]])
0.bias tensor([0., 0., 0.])
2.weight tensor([[-0.0127, -0.0007,  0.0093]])
2.bias tensor([0.])


In [44]:
def init_weight(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        #print('main:',tensor.uniform_(-10,10))
        #print(tensor.abs()>5)
        tensor*=(tensor.abs()>=5).float()
        #print(tensor)

In [46]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight(param)
        print(name,param.data)

0.weight tensor([[ 0.0000,  0.0000,  8.1440, -6.5662],
        [-0.0000, -0.0000, -7.3206,  7.8549],
        [ 5.4216,  7.4208,  7.9317,  7.3940]])
2.weight tensor([[ 0.0000, -7.6710, -0.0000]])


In [47]:
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data+=1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


In [50]:
linear = nn.Linear(1,1,bias=False)
net = nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [52]:
print(id(net[0])==id(net[1]))
print(id(net[0].weight)==id(net[1].weight))

True
True


In [54]:
x = torch.ones(1,1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.data)

tensor(9., grad_fn=<SumBackward0>)
tensor([[3.]])
